In [1]:
import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
/home/martin/anaconda3/envs/pi2/lib/python3.7/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [6]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..',params['model_dir'])

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [7]:
#preprocessing.download_data(params)
#preprocessing.extract_data(params)
#preprocessing.generate_spectogram_images(params)
#preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

## Segund Paso: instanciar el modelo

In [4]:
inputs = tf.keras.layers.Input(shape=(110,480, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

W0727 10:28:17.901544 140469068961600 deprecation.py:506] From /home/martin/anaconda3/envs/pi2/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model_architecture"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 240, 64)       9472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 240, 64)       256       
_________________________________________________________________
activation_3 (Activation)    (None, 55, 240, 64)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 48, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1584)              0         
_________________________________________________________________
dense (Dense)                (None, 12)                19020     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)        

## Tercer paso: crear el generador

In [9]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 2959 images.
Data in validation
Found 30 images.


## Cuarto paso: correr el modelo

In [10]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

steps_per_epoch = train_generator.n // params['batch_size']
validation_steps = val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback,tb_callback])

W0727 10:28:18.400288 140469068961600 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Epoch 1/25
296/296 [==============================] - 35s 117ms/step - loss: 0.5575 - sparse_categorical_accuracy: 0.8162 - val_loss: 0.5467 - val_sparse_categorical_accuracy: 0.9000
Epoch 2/25
296/296 [==============================] - 23s 77ms/step - loss: 0.1298 - sparse_categorical_accuracy: 0.9588 - val_loss: 0.3847 - val_sparse_categorical_accuracy: 0.9000
Epoch 3/25
296/296 [==============================] - 21s 72ms/step - loss: 0.0617 - sparse_categorical_accuracy: 0.9801 - val_loss: 0.3393 - val_sparse_categorical_accuracy: 0.9333
Epoch 4/25
296/296 [==============================] - 19s 64ms/step - loss: 0.0378 - sparse_categorical_accuracy: 0.9885 - val_loss: 0.5760 - val_sparse_categorical_accuracy: 0.9333
Epoch 5/25
295/296 [============================>.] - ETA: 0s - loss: 0.0620 - sparse_categorical_accuracy: 0.9790
Epoch 00005: saving model to ..\checkpoints\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
296/296 [==============================] - 21s 6

296/296 [==============================] - 23s 76ms/step - loss: 4.3017e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4962 - val_sparse_categorical_accuracy: 0.9333
Epoch 11/25
296/296 [==============================] - 22s 74ms/step - loss: 2.9919e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4619 - val_sparse_categorical_accuracy: 0.9333
Epoch 12/25
296/296 [==============================] - 21s 71ms/step - loss: 2.5148e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5127 - val_sparse_categorical_accuracy: 0.9333
Epoch 13/25
296/296 [==============================] - 20s 66ms/step - loss: 2.2128e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4667 - val_sparse_categorical_accuracy: 0.9333
Epoch 14/25
296/296 [==============================] - 19s 64ms/step - loss: 1.5802e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5241 - val_sparse_categorical_accuracy: 0.9333
Epoch 15/25
294/296 [============================>.] - ETA: 0s - loss: 1.3109

In [7]:
#fpr, tpr, thresholds = roc_curve(y, probs)

In [8]:
# Quintero paso: evaluar el modelo

In [9]:

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

Y_pred = net.predict_generator(val_generator, validation_steps)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(val_generator.data, y_pred))
print('Classification Report')
target_names = ['Cats', 'Dogs', 'Horse']
print(classification_report(val_generator.data, y_pred, target_names=["arranca", "acelera", "pita", "izquierda",
                                                                      "frena", "detente", "retrocede", "gira", "avanza", "derecha"]))

Confusion Matrix
[[0 0 1 0 0 0 0 1 1 0]
 [1 0 0 0 1 1 0 0 0 1]
 [1 0 1 0 0 0 1 0 0 0]
 [0 0 1 0 1 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 1 1]
 [0 0 0 0 1 0 1 0 0 1]
 [1 1 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0 0 2]]
Classification Report
              precision    recall  f1-score   support

     arranca       0.00      0.00      0.00         3
     acelera       0.00      0.00      0.00         4
        pita       0.25      0.33      0.29         3
   izquierda       0.00      0.00      0.00         2
       frena       0.00      0.00      0.00         4
     detente       0.00      0.00      0.00         3
   retrocede       0.00      0.00      0.00         3
        gira       0.00      0.00      0.00         2
      avanza       0.00      0.00      0.00         2
     derecha       0.33      0.50      0.40         4

   micro avg       0.10      0.10      0.10        30
   macro avg       0.06      0.08      0.07        30
weighted avg       0.07   